In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams['figure.figsize'] = (16, 10)
plt.rc('font', size=15)

In [10]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [11]:
import os

cur_dir = os.getcwd()
checkpoint_dir = os.path.join(cur_dir, 'checkpoints', 'mnist_cnn_seq')
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'mnist_cnn_seq')

In [12]:
from tensorflow.keras.utils import to_categorical

# MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalization
X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.

# Convert it to 4D array (or we can use np.expand_dims for dimension expansion)
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

# one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build dataset pipeline
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=100000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

In [13]:
def create_model():
    model = tf.keras.Sequential(name='CNN_Sequential')
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.keras.activations.relu,
                                     padding='SAME', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.keras.activations.relu,
                                     padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), activation=tf.keras.activations.relu,
                                     padding='SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding='SAME'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(10))
    return model

# Create model
model = create_model()
model.summary()

Model: "CNN_Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 4, 4, 128)     

In [14]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

# Gradient Function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Evaluation function
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_predict = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
    return accuracy

# Checkpoint
checkpoint = tf.train.Checkpoint(cnn=model)

In [16]:
for e in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_ds:
        grads = grad(model, images, labels)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_ds:
        acc = evaluate(model, images, labels)
        avg_test_acc = avg_test_acc + acc
        test_step += 1
    avg_test_acc = avg_test_acc / test_step
    
    print("Epoch: {}".format(e + 1),
          "loss: {:.8f}".format(avg_loss),
          "train acc: {:.4f}".format(avg_train_acc),
          "test acc: {:.4f}".format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

Epoch: 1 loss: 0.17913757 train acc: 0.9578 test acc: 0.9870
Epoch: 2 loss: 0.04863174 train acc: 0.9896 test acc: 0.9899
Epoch: 3 loss: 0.03315681 train acc: 0.9932 test acc: 0.9904
Epoch: 4 loss: 0.02572735 train acc: 0.9948 test acc: 0.9925
Epoch: 5 loss: 0.01919295 train acc: 0.9965 test acc: 0.9908
Epoch: 6 loss: 0.01603331 train acc: 0.9971 test acc: 0.9917
Epoch: 7 loss: 0.01319477 train acc: 0.9978 test acc: 0.9946
Epoch: 8 loss: 0.01167356 train acc: 0.9981 test acc: 0.9929
Epoch: 9 loss: 0.01066297 train acc: 0.9985 test acc: 0.9933
Epoch: 10 loss: 0.00778059 train acc: 0.9988 test acc: 0.9927
Epoch: 11 loss: 0.00698124 train acc: 0.9991 test acc: 0.9916
Epoch: 12 loss: 0.00653007 train acc: 0.9991 test acc: 0.9932
Epoch: 13 loss: 0.00609690 train acc: 0.9992 test acc: 0.9929
Epoch: 14 loss: 0.00555209 train acc: 0.9994 test acc: 0.9942
Epoch: 15 loss: 0.00506163 train acc: 0.9993 test acc: 0.9935


In [17]:
def create_model_functional():
    inputs = tf.keras.Input(shape=(28, 28, 1))
    conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME', 
                                   activation=tf.keras.activations.relu)(inputs)
    pool1 = tf.keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME',
                                   activation=tf.keras.activations.relu)(pool1)
    pool2 = tf.keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='SAME',
                                   activation=tf.keras.activations.relu)(pool2)
    pool3 = tf.keras.layers.MaxPool2D(padding='SAME')(conv3)
    pool3_flat = tf.keras.layers.Flatten()(pool3)
    dense4 = tf.keras.layers.Dense(units=256, activation=tf.keras.activations.relu)(pool3_flat)
    drop4 = tf.keras.layers.Dropout(rate=0.4)(dense4)
    logits = tf.keras.layers.Dense(units=10)(drop4)
    return tf.keras.Model(inputs=inputs, outputs=logits)

In [18]:
model = create_model_functional()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 128)         73856 

In [19]:
inputs = tf.keras.Input(shape=(28, 28, 256))
conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(1, 1), padding='SAME', activation=tf.keras.activations.relu)(inputs)
conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME', activation=tf.keras.activations.relu)(conv1)
conv3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(1, 1), padding='SAME')(conv2)
# skip connection
add3 = tf.keras.layers.add([conv3, inputs])
relu3 = tf.keras.activations.relu(add3)
model = tf.keras.Model(inputs=inputs, outputs=relu3)

In [20]:
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool1 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool3 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = tf.keras.layers.Flatten()
        self.dense4 = tf.keras.layers.Dense(units=256, activation=tf.keras.activations.relu)
        self.drop4 = tf.keras.layers.Dropout(rate=0.4)
        self.dense5 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [21]:
model = CNNModel()

In [22]:
model.build(input_shape=(1, 28, 28, 1))
model.summary()

Model: "cnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          multiple                  320       
                                                                 
 max_pooling2d_9 (MaxPooling  multiple                 0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          multiple                  18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          multiple                  73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  multiple                 0 

In [23]:
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool1 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool2 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3, 3), padding='SAME',
                                            activation=tf.keras.activations.relu)
        self.pool3 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = tf.keras.layers.Flatten()
        self.dense4 = tf.keras.layers.Dense(units=256, activation=tf.keras.activations.relu)
        self.drop4 = tf.keras.layers.Dropout(rate=0.4)
        self.dense5 = tf.keras.layers.Dense(units=10)
    
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [24]:
models = []
for m in range(3):
    models.append(CNNModel())

In [25]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

# Gradient Function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

In [26]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

def evaluate(models, images, labels):
    predicts = tf.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predicts += logits
    correct_predict = tf.equal(tf.argmax(predicts, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
    return accuracy

checkpoints = []
for model in models:
    checkpoints.append(tf.train.Checkpoint(cnn=model))

In [27]:
for e in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_ds:
        for model in models:
            grads = grad(model, images, labels)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            loss = loss_fn(model, images, labels)
            avg_loss += loss / 3
        acc = evaluate(models, images, labels)
        avg_train_acc += acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_ds:
        acc = evaluate(models, images, labels)
        avg_test_acc += acc
        test_step += 1
    avg_test_acc = avg_test_acc / test_step
    
    print("Epoch: {}".format(e + 1),
          "Loss: {:.8f}".format(avg_loss),
          "Train Accuracy: {:.4f}".format(avg_train_acc),
          "Test Accuracy: {:.4f}".format(avg_test_acc))
    
    for idx, checkpoint in enumerate(checkpoints):
        checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

Epoch: 1 Loss: 0.15972973 Train Accuracy: 0.9636 Test Accuracy: 0.9899
Epoch: 2 Loss: 0.03789086 Train Accuracy: 0.9933 Test Accuracy: 0.9905
Epoch: 3 Loss: 0.02561753 Train Accuracy: 0.9959 Test Accuracy: 0.9932
Epoch: 4 Loss: 0.01906417 Train Accuracy: 0.9974 Test Accuracy: 0.9942
Epoch: 5 Loss: 0.01414697 Train Accuracy: 0.9983 Test Accuracy: 0.9914
Epoch: 6 Loss: 0.01151887 Train Accuracy: 0.9988 Test Accuracy: 0.9942
Epoch: 7 Loss: 0.01049820 Train Accuracy: 0.9992 Test Accuracy: 0.9949
Epoch: 8 Loss: 0.00837994 Train Accuracy: 0.9993 Test Accuracy: 0.9938
Epoch: 9 Loss: 0.00733276 Train Accuracy: 0.9996 Test Accuracy: 0.9938
Epoch: 10 Loss: 0.00654636 Train Accuracy: 0.9996 Test Accuracy: 0.9943
Epoch: 11 Loss: 0.00558788 Train Accuracy: 0.9998 Test Accuracy: 0.9944
Epoch: 12 Loss: 0.00518030 Train Accuracy: 0.9998 Test Accuracy: 0.9938
Epoch: 13 Loss: 0.00446692 Train Accuracy: 0.9999 Test Accuracy: 0.9945
Epoch: 14 Loss: 0.00396374 Train Accuracy: 0.9998 Test Accuracy: 0.9944
E

In [28]:
from scipy import ndimage
import random

def data_augmentation(images, labels):
    aug_images = []
    aug_labels = []
    
    for image, label in zip(images, labels):
        aug_images.append(image)
        aug_labels.append(label)
        
        # Background image for filling empty pixel
        bg_value = np.median(image)
        for _ in range(4):
            # Rotation
            rot_image = ndimage.rotate(image, angle=random.randint(-15, 15), 
                                       reshape=False, cval=bg_value)
            # Shift
            shift_image = ndimage.shift(rot_image, shift=np.random.randint(-2, 2, 2), 
                                        cval=bg_value)
            
            aug_images.append(shift_image)
            aug_labels.append(label)
    aug_images = np.array(aug_images)
    aug_labels = np.array(aug_labels)
    return aug_images, aug_labels

In [29]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train, y_train = data_augmentation(X_train, y_train)

# Convert numpy float type and normalize it
X_train = X_train.astype(np.float32) / 255.
X_test = X_test.astype(np.float32) / 255.

# Convert it to 4D array (or we can use np.expand_dims for dimension expansion)
X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

# one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build dataset pipeline
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)

In [30]:
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=(3, 3), strides=1, padding='SAME'):
        super(ConvBNRelu, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, 
                                           strides=strides, padding=padding,
                                           kernel_initializer='glorot_normal')
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inputs, training=False):
        net = self.conv(inputs)
        net = self.bn(net)
        return tf.keras.activations.relu(net)
    
class DenseBNRelu(tf.keras.Model):
    def __init__(self, units):
        super(DenseBNRelu, self).__init__()
        self.dense = tf.keras.layers.Dense(units=units, kernel_initializer='glorot_normal')
        self.bn = tf.keras.layers.BatchNormalization()
        
    def call(self, inputs, training=False):
        net = self.dense(inputs)
        net = self.bn(net)
        return tf.keras.activations.relu(net)

In [31]:
class CNNModel(tf.keras.Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = ConvBNRelu(filters=32, kernel_size=(3, 3), padding='SAME')
        self.pool1 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = ConvBNRelu(filters=64, kernel_size=(3, 3), padding='SAME')
        self.pool2 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = ConvBNRelu(filters=128, kernel_size=(3, 3), padding='SAME')
        self.pool3 = tf.keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = tf.keras.layers.Flatten()
        self.dense4 = DenseBNRelu(units=256)
        self.drop4 = tf.keras.layers.Dropout(rate=0.4)
        self.dense5 = tf.keras.layers.Dense(units=10, kernel_initializer='glorot_normal')
        
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        return self.dense5(net)

In [32]:
models = []
for _ in range(5):
    models.append(CNNModel())

In [33]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

# Gradient Function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

# Evaluation Function
def evaluate(models, images, labels):
    predicts = tf.zeros_like(labels)
    for model in models:
        logits = model(images, training=False)
        predicts += logits
    correct_predict = tf.equal(tf.argmax(predicts, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
    return accuracy

In [34]:
lr_decay = tf.keras.optimizers.schedules.ExponentialDecay(learning_rate, 
                                          decay_steps=X_train.shape[0] / batch_size * 5 * 5,
                                          decay_rate=0.5,
                                          staircase=True)

# Optimizer with learning rate scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_decay)

# Checkpoint
checkpoints = []
for model in models:
    checkpoints.append(tf.train.Checkpoint(cnn=model))

In [ ]:
for e in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_ds:
        for model in models:
            grads = grad(model, images, labels)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            loss = loss_fn(model, images, labels)
            avg_loss += loss / 3
        acc = evaluate(models, images, labels)
        avg_train_acc += acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_ds:
        acc = evaluate(models, images, labels)
        avg_test_acc += acc
        test_step += 1
    avg_test_acc = avg_test_acc / test_step
    
    print("Epoch: {}".format(e + 1),
          "Loss: {:.8f}".format(avg_loss),
          "Train Accuracy: {:.4f}".format(avg_train_acc),
          "Test Accuracy: {:.4f}".format(avg_test_acc))
    
    for idx, checkpoint in enumerate(checkpoints):
        checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

Epoch: 1 Loss: 0.08116379 Train Accuracy: 0.9222 Test Accuracy: 0.9944
Epoch: 2 Loss: 0.03428531 Train Accuracy: 0.9963 Test Accuracy: 0.9947
